In [1]:
import torch
import transformers
import langchain
import chromadb
import xformers
import sentence_transformers
import tokenizers
import optimum
import auto_gptq
import unstructured
import accelerate
import pdf2image
import pypdf
import jq

In [2]:
import json
import torch
import jsonformer

from functools import partial
from threading import Thread
from typing import List, Mapping, Optional, Any, Dict
from pydantic import BaseModel, Field, validator

In [3]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.schema import BaseOutputParser
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.output_parsers import PydanticOutputParser,OutputFixingParser,RetryWithErrorOutputParser
from langchain.output_parsers.format_instructions import PYDANTIC_FORMAT_INSTRUCTIONS
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, TextStreamer, TextIteratorStreamer

In [4]:
# a class to create a question answering system based on information retrieval
from langchain.chains import RetrievalQA
# a class to create text embeddings using HuggingFace templates
from langchain.embeddings import HuggingFaceEmbeddings
# a class for splitting text into fixed-sized chunks with an optional overlay
from langchain.text_splitter import RecursiveCharacterTextSplitter
# a class for loading PDF documents from a directory
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path

In [5]:
 # download the embeddings to use to represent text chunks in a vector space, using the pre-trained model "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
import os

file_path = 'styleDB'

if not os.path.exists(file_path):
    

In [8]:
from langchain.document_loaders import JSONLoader
import os

file_path = "./styleDB"

if os.path.exists(file_path):
    db = Chroma(persist_directory=file_path, embedding_function=embeddings)
else:
    loader = JSONLoader("./total_keywords_w_artist.json",
    jq_schema='.data[]', text_content=False)

    data = loader.load()
    print(data)
    
    # split the extracted data into text chunks using the text_splitter, which splits the text based on the specified number of characters and overlap
    style_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    style_chunks = text_splitter.split_documents(data)
    
    db = Chroma(persist_directory="styleDB", embedding_function=embeddings)
    db.add_documents(documents=style_chunks)

[Document(page_content='{"description": "The JSON file contains style-modifiers list categorized as \'color\', \'dimensionality\', \'light\', \'perspective\', and \'style\'. In each category, modifiers are separated by commas.", "modifiers": [{"color": ["White", "Black", "Brown", "Light Gray", "Gray", "Dark Gray", "Maroon", "Red", "Orange", "Yellow", "Lime", "Green", "Cyan", "Teal", "Blue", "Indigo", "Purple", "Violet", "Fuchsia", "Magenta", "Pink", "Tan", "Beige", "Blush", "Scarlet", "Olive Green", "Chartreuse", "Turquoise", "Aqua", "Azure", "Dark White", "Dark Brown", "Dark Maroon", "Dark Red", "Dark Orange", "Dark Yellow", "Dark Lime", "Dark Green", "Dark Cyan", "Dark Blue", "Dark Purple", "Dark Magenta", "Dark Pink", "Light Black", "Light Brown", "Light Maroon", "Light Red", "Light Orange", "Light Yellow", "Light Lime", "Light Green", "Light Cyan", "Light Blue", "Light Purple", "Light Magenta", "Light Pink", "Vivid Brown", "Vivid Maroon", "Vivid Red", "Vivid Orange", "Vivid Yellow"

In [17]:
db.get()

{'ids': ['966a324a-e91c-11ee-b3fd-391e7d7b1785',
  '966a324b-e91c-11ee-b3fd-391e7d7b1785',
  '966a324c-e91c-11ee-b3fd-391e7d7b1785',
  '966a324d-e91c-11ee-b3fd-391e7d7b1785',
  '966a324e-e91c-11ee-b3fd-391e7d7b1785',
  '966a324f-e91c-11ee-b3fd-391e7d7b1785',
  '966a3250-e91c-11ee-b3fd-391e7d7b1785',
  '966a3251-e91c-11ee-b3fd-391e7d7b1785',
  '966a3252-e91c-11ee-b3fd-391e7d7b1785',
  '966a3253-e91c-11ee-b3fd-391e7d7b1785',
  '966a3254-e91c-11ee-b3fd-391e7d7b1785',
  '966a3255-e91c-11ee-b3fd-391e7d7b1785',
  '966a3256-e91c-11ee-b3fd-391e7d7b1785',
  '966a3257-e91c-11ee-b3fd-391e7d7b1785',
  '966a3258-e91c-11ee-b3fd-391e7d7b1785'],
 'embeddings': None,
 'metadatas': [{'seq_num': 1,
   'source': '/home/a2070064/langchain/total_keywords_w_artist.json'},
  {'seq_num': 1,
   'source': '/home/a2070064/langchain/total_keywords_w_artist.json'},
  {'seq_num': 1,
   'source': '/home/a2070064/langchain/total_keywords_w_artist.json'},
  {'seq_num': 1,
   'source': '/home/a2070064/langchain/total_ke

In [18]:
from auto_gptq import AutoGPTQForCausalLM

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

INFO - The layer lm_head is not quantized.
Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not supported with use_triton=False.


In [19]:
DEFAULT_SYSTEM_PROMPT = """
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [20]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [21]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [22]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [23]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
"""
{context}

Instruction: Give style modifiers for generating the image of the input TEXT.
1. From the given context, select style modifiers to improve the emotion expression, aesthetics and authenticity of the final image.
2. Output style modifiers in 3 or less.

TEXT: {question}
"""
,
    system_prompt=SYSTEM_PROMPT,
)

In [24]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [27]:
import pandas as pd

df = pd.read_csv('main_data.csv')

In [28]:
df['prompt'] = df['emotion'] +', ' + df['original_text']
df['answer'] = ""
df.head()

In [30]:
for index, row in df.iloc[:].iterrows():
    answer_format = '\n* Use this format(JSON) for the answer {\"modifiers\": []}.'
    text = row['prompt']
    df.at[index, 'answer'] = qa_chain(text+answer_format).get("result")
    df.to_csv('./S2_answer.csv', index=False)

/home/a2070064/langchain/lib/python3.8/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/a2070064/langchain/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/a2070064/langchain/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Based on the given text, here are three style modifiers that could be used to generate an image that accurately represents the emotions expressed in the text:

1. Desaturated colors: To convey the sadness and devastation mentioned in the text, desaturated colors such as muted blues, grays, and purples could be used. This would create a somber and melancholic atmosphere that aligns with the emotional tone of the text.
2. Soft brush strokes: Using soft brush strokes with a bit of texture could add a sense of vulnerability and fragility to the image, emphasizing the struggles and challenges mentioned in the text. This technique would also help to convey a sense of delicacy and sensitivity.
3. Warm lighting: Warm lighting with a yellowish or orange tint could be used to represent the hope and resilience expressed in the text. This lighting would add a sense of comfort and optimism to the image, balancing out the sadness and devastation described in the text.

Therefore, the output style m